In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
if '../src' not in sys.path:
  sys.path.append('../src')
import os
os.environ["BNB_CUDA_VERSION"] = "115"
import numpy as np
from llm_compressor import AECompressorLLM
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
from pathlib import Path
import pickle
error0 = pickle.load(open('error_inst.pkl', 'rb'))
error2 = pickle.load(open("error_inst-2.pkl", "rb"))

In [5]:
dbg = {}
error_inst = error0
data_ids = error_inst["data_ids"]
next_token_probs = error_inst["probs"]
compressor = AECompressorLLM()

msg = compressor.compress(data_ids, next_token_probs)
recon = compressor.decompress(msg, len(data_ids), next_token_probs)
## uniform probs baseline
# msg = compressor.compress(data_ids, uniform_nt_probs)
# recon = compressor.decompress(msg, len(data_ids), uniform_nt_probs)

assert all(a==b for a, b in zip(recon, data_ids))
msg_len = len(msg)
data_len = len(data_ids) * 16
print(f"message length: {msg_len} bits")
print(f"data length: {data_len} bits")
print(f"compress ratio: {msg_len/data_len:.4f}")


message length: 621 bits
data length: 688 bits
compress ratio: 0.9026


In [30]:
freqs = compressor.model.dbg["freqs"]

In [37]:
np.cumsum(freqs.astype(np.int64))

array([        79,       2752,       2792, ..., 4453108732, 4453110030,
       4453111571])

In [16]:
np.cumsum([1,2,3])

array([1, 3, 6])

In [26]:
# newlow = low + symlow * range // total
# newhigh = low + symhigh * range // total - 1

dbg = compressor.encoder.dbg
prob = dbg["prob"]
sym = dbg["symbol"]
total = dbg["total"]
r = dbg["range"]
prob[sym], total, r

([5978989, 5979053), 9985536, 4294967296)

In [24]:
1 / total

0.971749506140457

In [8]:
import zlib
zmsg = zlib.compress(test_text.encode())

In [9]:
len(zmsg)/len(test_text.encode())

1.2758620689655173

## Appendix

In [10]:
## clear float2repr implmentation by Copilot
import struct

def float_repr(num):
    # pack the float into a bytes object
    packed = struct.pack('f', num)
    
    # unpack the bytes object to get the exponent and fractional part
    bits = struct.unpack('I', packed)[0]    
    sign = bits >> 31
    exp = (bits >> 23) & 0xff
    frac = bits & 0x7fffff
    
    # convert the exponent to a signed integer
    if exp == 0:
        exp = -126
    else:
        exp -= 127
    
    # convert the fractional part to a float
    frac = float(frac) / (1 << 23)
    
    # apply the sign, exponent, and fractional part to get the final representation
    print("sign: ", sign)
    print("exp: ", exp)
    print("frac: ", frac)
    assert (-1)**sign * (1 + frac) * 2**exp == num
